In [1]:
# Import packages
from unityagents import UnityEnvironment
import numpy as np
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

# Set enviroment
env = UnityEnvironment(file_name="/media/riley/Work/PersonalProjects/AI/Deep_Reinforcement_Learning/Project1/Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [2]:
# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


# DQN

In [4]:
#from dqn_agent import Agent
from agent import DQNPRBAgent
from agent import DQNAgent

agent = DQNAgent(state_size=37, action_size=4, seed=0, use_dueling=True, use_double=True)

cuda:0


In [5]:
def dqn(n_episodes=3000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Train the Deep Q network."""
    highest_score = 0
    scores = []
    scores_windows = deque(maxlen=100)
    eps = eps_start
    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        while True:
            action = agent.act(state, eps)
            # send the action to the environment
            env_info = env.step(action)[brain_name]
            # get the next state
            next_state = env_info.vector_observations[0]
            # get the reward
            reward = env_info.rewards[0]
            # see if episode has finished
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            # update the score
            score += reward
            # roll over the state to next time step
            state = next_state
            
            if done:
                break
        scores_windows.append(score)
        scores.append(score)
        eps = max(eps_end, eps * eps_decay)
        if np.mean(scores_windows) > highest_score:
            highest_score = np.mean(scores_windows)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_windows)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_windows)))
        if np.mean(scores_windows)>=13.0:
            if np.mean(scores_windows) == highest_score:
                print('\tModel Saved')
                torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 0.67
Episode 171	Average Score: 2.41

KeyboardInterrupt: 

In [ ]:
import time
checkpoint = torch.load('checkpoint.pth')
agent = Agent(state_size=37, action_size=4, seed=0)
agent.qnetwork_local.load_state_dict(checkpoint)


env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]
score = 0
while True:
    time.sleep(0.015)
    action = agent.act(state, 0)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if done:
        break
    
print("Score: {}".format(score))

In [ ]:
env.close()